# Data Prepration

In [1]:
from prepross import *
import numpy as np
import os
import dataloader
from dataloader import Vocab
pd.options.display.max_rows = 100
from embedding import Doc2VecEmbedding
import pickle
import numpy as np
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mehrazin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mehrazin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
class Parse():
    def __init__(self):
        self.test_mode = True
        self.load_vocab = True
arg = Parse()

In [3]:
config = Config(arg)

AttributeError: 'Parse' object has no attribute 'exp_id'

In [ ]:
config.exp_dir

In [ ]:
config.Dataset_dir

In [ ]:
config.device

In [ ]:
# df = Data_handler(config, pass_col = ['lyrics'])
# df.data = df.get_masked_data(users = df.users[:3], fracs = [0.1,0.2,0.1])
# df.data = create_session(df.data, config)
# df.data = clean_data(df.data, config)
# df.data = cut_sessions(df.data, config)
# config.clean_mode = ['rm_small_sub_session']
# df.data = clean_data(df.data, config)
# df.data

# Model Prepration

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from dataloader import Music_data, create_data_loader
import torch.optim as optim
import torch.nn.functional as F
import Model
import random
from Model import create_embeding, Seq2Seq, Encoder, Decoder

In [ ]:
seed = 0
"""Set seed"""
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
config.num_epochs = 1000
config.batch_size = 50
config.learning_rate = 0.001

In [ ]:
embedding, dim, _ = Model.create_embeding(config)
train_iterator = create_data_loader(config, 'train', shuffle = True)
test_iterator = create_data_loader(config, 'test', shuffle = True)
valid_iterator = create_data_loader(config, 'valid', shuffle = True)
itr = iter(train_iterator)
# df = Music_data(config, 'train')

In [ ]:
enc = Encoder(config).to(config.device)
dec = Decoder(config).to(config.device)
model = Seq2Seq(config, enc, dec).to(config.device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
pad_idx = config.pad_index
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [ ]:
# Tensorboard to get nice loss plot
writer = SummaryWriter(f"runs/loss_plot")
step = 0

In [ ]:
# Overfit a single batch
batch = next(iter(train_iterator))